In [106]:
#imports block

import pandas as pd
import numpy as np
import string, re

from sklearn.model_selection import train_test_split

import nltk
import spacy
#nltk.download()
from nltk import word_tokenize, pos_tag
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [107]:
#Reading Input
fake_df = pd.read_csv('Fake.csv')
true_df = pd.read_csv('True.csv')

In [108]:
true_df.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [109]:
fake_df.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,31-Dec-17
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-Dec-17
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-Dec-17
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,29-Dec-17
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-Dec-17


In [110]:
#Exploratory Data Analysis

In [111]:
fake_df['class'] = 0
true_df['class'] = 1
#Concat Full Data
df = pd.concat([fake_df, true_df])
#Shuffle the Data
df = df.sample(frac = 1)
df.reset_index(inplace = True)
#Filter Required columns
df= df[['text','class']]
df

,text,class
0,KABUL (Reuters) - A senior U.S. commander in A...,1
1,WASHINGTON (Reuters) - Two U.S. Senators urged...,1
2,Trump is still beating his widespread voter f...,0
3,The bashing of Donald Trump is getting so old ...,0
4,You know the age-old political trope that poli...,0
...,...,...
44935,A teenage girl who fought back when she was al...,0
44936,PHOENIX (Reuters) - Two National Football Leag...,1
44937,LONDON (Reuters) - Prime Minister Theresa May ...,1
44938,WASHINGTON (Reuters) - While he has swallowed ...,1


In [112]:
#Train Test Split
#Text Processing pipeline: Lower case + remove punctuation + Tokenization + Lemmatization + Stop words Removal
#Convert these to Vectors using BOW or TF-IDF or Embeddings(Glove, fasttext, Word2Vec)
#Make Baseline ML models(Log Reg, SVM, Naive Bayes, Decision Trees, RF, XGBoost Classifier)
#Hyperparameter Tuning of above models
#Make DL models using CNN, LSTM's, BERT/RoBERTa

In [113]:
#Text preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    return phrase


def clean_text(text):

    #lower case text
    text = text.lower().strip()

    #remove http links
    text = re.sub(r"http\S+", "", text)
    #expanding some words
    text = decontracted(text)
    #remove special character
    #text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = re.sub('[^A-Za-z]+', ' ', text)
    #remove numbers
    #text = re.sub("\S*\d\S*", "", text).strip()
    
    #text = text.strip()
    return text



Normalisation Required??? 

Normalisation performs:

    Converting dates to text
    Numbers to text
    Currency/Percent signs to text
    Expanding of abbreviations (content dependent) NLP - Natural Language Processing, Neuro-linguistic programming, Non-Linear programming
    Spelling mistakes correction

In [114]:
#from normalise.normalisation import normalise

#def normalization(tokens):
#    user_abbr = { 'NLP' : 'Natural Language Processing'}
#    normalised_tokens = normalise(tokens, user_abbrevs = user_abbr, verbose = False)
#    return normalised_tokens

def remove_stop_words(words):
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

def stemming(words):
    porter = PorterStemmer()
    stemmed_words = [porter.stem(word) for word in words]
    return stemmed_words

def remove_punctuation(text):
    text_processed = "".join([char for char in text if char not in string.punctuation])
    return text_processed


#Lower text: text = text.lower()
#Tokenization Code:  words = word_tokenize(text)
#Parts Of Speech Tagging: words = pos_tag(words)

In [115]:
custom_stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't",'no','nor','not'])

def spacy_process(text):
    print(text)
    print(100*'*')
    text = clean_text(text)
    doc = nlp(text)
    
    #Tokenization and lemmatization are done with the spacy nlp pipeline commands
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
    print("Tokenize+Lemmatize:")
    print(lemma_list)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        if word not in custom_stopwords:
            filtered_sentence.append(word)
    
    #Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)
    print(" ")
    print("Remove stopword & punctuation: ")
    print(filtered_sentence)

first_row = fake_df.iloc[0]
spacy_process(first_row['text'])

Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t eve